1 imagenet256のラベル付けを変更する 

2 imagenet_class_index_lower.jsonにimagenet-256のデータセットのラベルと会うように文字列を修正（手動＋コード）

3 事前学習済みのresnet50を用いてimagenet-256のデータセットの画像分類

In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm import tqdm
import json


In [2]:
# 1. 前処理
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 2. データセット
data_dir = "./imagenet-256"
val_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# 3. モデル
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()


/home/gotou/miniconda3/envs/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gotou/miniconda3/envs/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# 4. ImageNet クラスインデックスを読み込み
with open("imagenet_class_index_lower.json", "r") as f:
    class_idx = json.load(f)

# name → index の辞書を作成（空白はアンダースコアに変換）
name_to_idx = {}
for k, v in class_idx.items():
    idx = int(k)
    synset, name = v
    # "afghan hound" → "afghan_hound"
    norm_name = name.replace(" ", "_")
    name_to_idx[norm_name] = idx

# abc順 (val_dataset.classes) → ImageNetインデックス
abc2idx = {}
missing = []
for cls_name in val_dataset.classes:
    if cls_name in name_to_idx:
        abc2idx[cls_name] = name_to_idx[cls_name]
    else:
        missing.append(cls_name)

print(f"[INFO] マッチできたクラス数: {len(abc2idx)}/{len(val_dataset.classes)}")
if missing:
    print(f"[WARN] マッチできなかったクラス例: {missing[:10]} ...")


[INFO] マッチできたクラス数: 999/1000
[WARN] マッチできなかったクラス例: ['tank_suit'] ...


In [ ]:

# 5. 精度計算
correct1, correct5, total = 0, 0, 0

# ...existing code...

correct_top1_paths = []
correct_top5_paths = []

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating", unit="batch")):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        valid_indices = []
        valid_labels_idx = []
        valid_paths = []
        for i, l in enumerate(labels.cpu()):
            cls_name = val_dataset.classes[l]
            if cls_name in abc2idx:
                valid_indices.append(i)
                valid_labels_idx.append(abc2idx[cls_name])
                img_idx = batch_idx * val_loader.batch_size + i
                img_path, _ = val_dataset.samples[img_idx]
                valid_paths.append(img_path)

        if not valid_indices:
            continue

        valid_images = images[valid_indices]
        valid_labels_idx = torch.tensor(valid_labels_idx).to(device)
        valid_outputs = outputs[valid_indices]

        # top-1
        _, pred1 = valid_outputs.max(1)
        correct_mask1 = pred1.eq(valid_labels_idx)
        for idx, is_correct in enumerate(correct_mask1.cpu()):
            if is_correct:
                correct_top1_paths.append(valid_paths[idx])

        # top-5
        _, pred5 = valid_outputs.topk(5, 1, True, True)
        correct_mask5 = pred5.eq(valid_labels_idx.view(-1,1)).any(dim=1)
        for idx, is_correct in enumerate(correct_mask5.cpu()):
            if is_correct:
                correct_top5_paths.append(valid_paths[idx])

        correct1 += correct_mask1.sum().item()
        correct5 += correct_mask5.sum().item()
        total += len(valid_indices)

print(f"Top-1 Accuracy: {100 * correct1 / total:.2f}%")
print(f"Top-5 Accuracy: {100 * correct5 / total:.2f}%")

# 正解画像パスをファイルに保存
with open("correct_top1_images.txt", "w") as f:
    for path in correct_top1_paths:
        f.write(path + "\n")

with open("correct_top5_images.txt", "w") as f:
    for path in correct_top5_paths:
        f.write(path + "\n")

print(f"Top-1で正解した画像数: {len(correct_top1_paths)}")
print(f"Top-5で正解した画像数: {len(correct_top5_paths)}")
# ...existing code...


[INFO] マッチできたクラス数: 999/1000
[WARN] マッチできなかったクラス例: ['tank_suit'] ...


Evaluating: 100%|██████████| 4218/4218 [12:06<00:00,  5.81batch/s]


Top-1 Accuracy: 85.51%
Top-5 Accuracy: 97.17%
Top-1で正解した画像数: 461169
Top-5で正解した画像数: 524073


In [4]:
import torch
from PIL import Image
from tqdm import tqdm

batch_size = 32  # メモリに合わせて調整
epsilon = 0.03

# 正解画像パスの読み込み
with open("correct_top1_images.txt") as f:
    correct_paths = [line.strip() for line in f]

adv_correct1, adv_correct5, adv_total = 0, 0, 0

model.eval()
for i in tqdm(range(0, len(correct_paths), batch_size), desc="FGSM Attack (batch)"):
    batch_paths = correct_paths[i:i+batch_size]
    images = []
    labels = []
    for path in batch_paths:
        img = Image.open(path).convert("RGB")
        input_tensor = transform(img)
        images.append(input_tensor)
        cls_name = path.split("/")[-2]
        if cls_name in abc2idx:
            labels.append(abc2idx[cls_name])
        else:
            labels.append(-1)  # 無効値

    # 無効なラベルを除外
    valid_indices = [j for j, l in enumerate(labels) if l >= 0]
    if not valid_indices:
        continue
    images = torch.stack([images[j] for j in valid_indices]).to(device)
    labels = torch.tensor([labels[j] for j in valid_indices]).to(device)
    images.requires_grad = True

    # FGSM
    output = model(images)
    loss = torch.nn.functional.cross_entropy(output, labels)
    model.zero_grad()
    loss.backward()
    data_grad = images.grad.data
    perturbed = images + epsilon * data_grad.sign()
    perturbed = torch.clamp(perturbed, 0, 1)

    # 再分類
    output_adv = model(perturbed)
    _, pred1 = output_adv.max(1)
    _, pred5 = output_adv.topk(5, 1, True, True)

    adv_total += len(labels)
    adv_correct1 += pred1.eq(labels).sum().item()
    adv_correct5 += sum([labels[j].item() in pred5[j].cpu().numpy() for j in range(len(labels))])

print(f"[FGSM] Top-1 Accuracy: {100 * adv_correct1 / adv_total:.2f}%")
print(f"[FGSM] Top-5 Accuracy: {100 * adv_correct5 / adv_total:.2f}%")

FGSM Attack (batch):   0%|          | 0/14412 [00:00<?, ?it/s]

FGSM Attack (batch): 100%|██████████| 14412/14412 [49:34<00:00,  4.84it/s]

[FGSM] Top-1 Accuracy: 27.85%
[FGSM] Top-5 Accuracy: 50.85%
